In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import string

import pydot
from networkx.drawing.nx_pydot import graphviz_layout

In [ ]:
np.random.seed(1)

G_edited = nx.disjoint_union_all([nx.complete_graph(n) for n in [3, 4]])
G_edited = nx.relabel_nodes(G_edited, dict(zip(G_edited, string.ascii_lowercase)))
edits = [("b", "g"), ("d", "e")]

deletions = set(edits) - set(G_edited.edges())
inserts = set(edits) - deletions

G = G_edited.copy()
G.add_edges_from(deletions)
G.remove_edges_from(inserts)

graphs = [
    ("ex-0", G, set(G.edges()) - set(inserts), [], [], [], []),
    ("ex-1", G, set(G.edges()) - set(inserts), [], [], [("b", "e")], [("b", "e"), ("b", "g"), ("e", "g")]),
    ("ex-2", G, set(G.edges()) - set(edits), [], deletions, [("b", "e")], [("b", "e"), ("b", "g"), ("e", "g")]),
    ("ex-3", G, set(G.edges()) - set(edits), [], [], [], []),
    ("ex-4", G, set(G.edges()) - set(edits), [], [], [("d", "e")], [("d", "e"), ("e", "g"), ("d", "g")]),
    ("ex-5", G, set(G.edges()) - set(deletions), [("d", "e")], [], [], [("d", "e"), ("e", "g"), ("d", "g")]),
    ("ex-6", G, G_edited.edges(), (), (), (), ()),
    ("P3-editing-example-G", G, set(G.edges()), [("d", "e")], [("b", "g")], (), ())
]

for name, G, unedited, inserted, deleted, non_edges, fat in graphs:
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.margins(0.15)
    ax.set_aspect("equal")
    ax.axis(False)

    def width(edges):
        return [3 if e in fat else 2 if e in (set(inserted) | set(deleted)) else 1 for e in edges]

    pos = graphviz_layout(G, prog="neato")
    nx.draw_networkx_nodes(G, pos, node_color="lightgrey", node_size=400, ax=ax)
    nx.draw_networkx_labels(G, pos, labels={v: f"${v}$" for v in G}, ax=ax)

    edge_lists = [
        (unedited,  "black", "solid"),
        (inserted,  "limegreen", "dashed"),
        (deleted,   "red",   "solid"),
        (non_edges, "grey", "dashed")
    ]

    for edgelist, edge_color, style in edge_lists:
        nx.draw_networkx_edges(G, pos, edgelist=edgelist, edge_color=edge_color, style=style, ax=ax, width=width(edgelist))

    fig.tight_layout()
    #plt.show()
    plt.savefig(f"{name}.pdf", bbox_inches="tight", pad_inches=0)
